## Import Libraries

In [18]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from casadi import Function

## Working with Symbolic Variables

In [19]:
# Defining First transformation
theta_1 = ca.MX.sym('theta_1', 1, 1)
n_1 = ca.MX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
tx_1 = ca.MX.sym("x_1", 1, 1)
t_1 = ca.vertcat(0.0, 0.0, 0.0, 0.0)

theta_2 = ca.MX.sym('theta_2', 1, 1)
n_2 = ca.MX([0.0, 0.0, 1.0])
q_2 = ca.vertcat(ca.cos(theta_2/2), ca.sin(theta_2/2)@n_2)
tx_2 = ca.MX.sym("x_2", 1, 1)
t_2 = ca.vertcat(0.0, tx_2, 0.0, 0.0)


aux_symbolic_total = ca.vertcat(theta_1, theta_2, tx_2)
symbolic_actions = ca.vertcat(theta_1, theta_2)

In [20]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)

In [21]:
Function_DualQuat_total = Function('DualQuat_total', [aux_symbolic_total], [Q1.get[:, 0]])
Function_DualQuat_total_trans = Function('DualQuat_total_trans', [aux_symbolic_total], [Q1.get_trans.get[:, 0]])
Function_DualQuat_total_quat = Function('DualQuat_total_quat', [aux_symbolic_total], [Q1.get_quat.get[:, 0]])

In [22]:
input_t = np.array([np.pi/2, 1.0])

trans_t = Function_DualQuat_total_trans(input_t)
quat_t = Function_DualQuat_total_quat(input_t)
Quat_t = Function_DualQuat_total(input_t)

In [23]:
trans_t

DM([-0, 1, 0, 0])

In [24]:
quat_t

DM([0.707107, 0, 0, 0.707107])

In [25]:
Quat_t

DM([0.707107, 0, 0, 0.707107, -0, 0.353553, -0.353553, 0])

In [31]:
input_t = np.array([4.97, 1.0])
trans = Function_DualQuat_total_trans(input_t)
trans

DM([0, 1, 0, 0])

### Calculating Jacobians

In [26]:
J = ca.jacobian(Q1.get[:, 0], symbolic_actions)

In [27]:
Function_J = Function("Function_F", [aux_symbolic_total], [J])

In [28]:
J.shape

(8, 1)